### Make sure you have mlflow installed in your environment:
```
pip install mlflow
```

In [ ]:
# import subprocess

# subprocess.run(["mlflow", "server", "--host", "127.0.0.1", "--port", "8080"])


In [1]:
from sklearn.datasets import make_regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature

# mlflow.set_tracking_uri(uri="http://127.0.0.1:8080")

with mlflow.start_run() as run:
    X, y = make_regression(n_features=4, n_informative=2, random_state=0, shuffle=False)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    params = {"max_depth": 2, "random_state": 42}
    model = RandomForestRegressor(**params)
    model.fit(X_train, y_train)

    # Infer the model signature
    y_pred = model.predict(X_test)
    signature = infer_signature(X_test, y_pred)

    # Log parameters and metrics using the MLflow APIs
    mlflow.log_params(params)
    mlflow.log_metrics({"mse": mean_squared_error(y_test, y_pred)})

    # Log the sklearn model and register as version 1
    mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path="sklearn-model",
        signature=signature,
        registered_model_name="sk-learn-random-forest-reg-model",
    )


Registered model 'sk-learn-random-forest-reg-model' already exists. Creating a new version of this model...
Created version '3' of model 'sk-learn-random-forest-reg-model'.


In [2]:
import mlflow.pyfunc

model_name = "sk-learn-random-forest-reg-model"
model_version = 1

model = mlflow.pyfunc.load_model(model_uri=f"models:/{model_name}/{model_version}")

# model.predict(data)

In [3]:
model

mlflow.pyfunc.loaded_model:
  artifact_path: sklearn-model
  flavor: mlflow.sklearn
  run_id: f659067ca9df4e4e8e9f60cf9a991302

In [5]:
from mlflow.tracking import MlflowClient

client = MlflowClient()

# List all versions of the model
model_name = "sk-learn-random-forest-reg-model"
versions = client.get_registered_model(model_name).latest_versions

print("Available Versions:")
for version in versions:
    print(f"Version: {version.version}, Stage: {version.current_stage}, Source: {version.source}")


Available Versions:
Version: 3, Stage: None, Source: file:///g:/Programming_Playground/Python_Programming/MLOps/Remote/MLOps_Playground/Excersise_4/mlruns/0/dc49fa5a5ded411689261f96239bbe96/artifacts/sklearn-model


In [6]:
import mlflow.sklearn
from mlflow.models import infer_signature
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split

# Create a dataset
X, y = make_regression(n_features=4, n_informative=2, random_state=0, shuffle=False)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a new model
params = {"max_depth": 3, "random_state": 42}
model = RandomForestRegressor(**params)
model.fit(X_train, y_train)

# Infer the model signature
y_pred = model.predict(X_test)
signature = infer_signature(X_test, y_pred)

# Log and register as version 2
with mlflow.start_run():
    mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path="sklearn-model-v2",
        signature=signature,
        registered_model_name="sk-learn-random-forest-reg-model",
    )


Registered model 'sk-learn-random-forest-reg-model' already exists. Creating a new version of this model...
Created version '4' of model 'sk-learn-random-forest-reg-model'.


In [7]:
client = MlflowClient()
versions = client.get_registered_model(model_name).latest_versions

for version in versions:
    print(f"Version: {version.version}, Stage: {version.current_stage}")


Version: 4, Stage: None


In [8]:
## If you had also created the second version:
model_name = "sk-learn-random-forest-reg-model"
model_version = 2

model = mlflow.pyfunc.load_model(model_uri=f"models:/{model_name}/{model_version}")

In [9]:
model

mlflow.pyfunc.loaded_model:
  artifact_path: sklearn-model-v2
  flavor: mlflow.sklearn
  run_id: 98e11dfcb1654870a47b2f531333f2a6

In [10]:
# Example input for prediction
import pandas as pd
data = pd.DataFrame(X_test)
predictions = model.predict(data)
print(predictions)

[-19.68300689  -9.76013937  39.74605937 -48.14179744 -41.45114231
 -16.46170476  39.70297597 -43.27469031 -51.06122427  25.78209347
 -17.29540579 -26.89415385  -6.46354864 -12.72026491   9.40222502
   9.6741405  -15.17570172   9.40222502 -23.37170498 -13.39018078]
